### Import Libraries

In [131]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.cm as cm
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.models import Model
from keras import layers, losses
from sklearn import preprocessing


from keras.models import Sequential
from keras.layers import InputLayer, LSTM, Dense, RepeatVector, TimeDistributed
from keras.utils import plot_model

In [2]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

### Load data

In [3]:
DATA_DIR_SLOW = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaSlow.npy"
DATA_DIR_NORMAL = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaNormal.npy"
DATA_DIR_C_NAMES = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaColumnNames.npy"

# Set style for matplotlib
plt.style.use("Solarize_Light2")

In [6]:
data_normal = np.load(DATA_DIR_NORMAL)
data_slow = np.load(DATA_DIR_SLOW)
column_names = np.load(DATA_DIR_C_NAMES)

# data_normal = data_normal.reshape((data_normal.shape[0], data_normal.shape[1], 1))
# data_slow = data_slow.reshape((data_slow.shape[0], data_slow.shape[1], 1))

In [5]:
column_names

array(['action', 'machine_nameKuka Robot_apparent_power',
       'machine_nameKuka Robot_current',
       'machine_nameKuka Robot_frequency',
       'machine_nameKuka Robot_phase_angle',
       'machine_nameKuka Robot_power',
       'machine_nameKuka Robot_power_factor',
       'machine_nameKuka Robot_reactive_power',
       'machine_nameKuka Robot_voltage', 'sensor_id1_AccX',
       'sensor_id1_AccY', 'sensor_id1_AccZ', 'sensor_id1_GyroX',
       'sensor_id1_GyroY', 'sensor_id1_GyroZ', 'sensor_id1_q1',
       'sensor_id1_q2', 'sensor_id1_q3', 'sensor_id1_q4',
       'sensor_id1_temp', 'sensor_id2_AccX', 'sensor_id2_AccY',
       'sensor_id2_AccZ', 'sensor_id2_GyroX', 'sensor_id2_GyroY',
       'sensor_id2_GyroZ', 'sensor_id2_q1', 'sensor_id2_q2',
       'sensor_id2_q3', 'sensor_id2_q4', 'sensor_id2_temp',
       'sensor_id3_AccX', 'sensor_id3_AccY', 'sensor_id3_AccZ',
       'sensor_id3_GyroX', 'sensor_id3_GyroY', 'sensor_id3_GyroZ',
       'sensor_id3_q1', 'sensor_id3_q2', 'sensor_id

## Dataset Setup

In [137]:
scaler = preprocessing.StandardScaler().fit(data_normal)
data_train_scaled = scaler.transform(data_normal)
data_test_scaled = scaler.transform(data_slow[:,:-1])

In [140]:
#windows of 50 with stride 1
width_window = 50
data_train = np.array([data_train_scaled[i:i+width_window,:] for i in range(0, len(data_train_scaled)-width_window)])
data_test = np.array([data_test_scaled[i:i+width_window,:] for i in range(0, len(data_test_scaled)-width_window)])
print(f"train data shape: {data_train.shape} \n test data shape: {data_test.shape} ")

train data shape: (233742, 50, 86) 
 test data shape: (41488, 50, 86) 


In [61]:
# data_train = data_train.reshape((data_train.shape[0], data_train.shape[1], data_train.shape[2], 1))
# data_test = data_test.reshape((data_test.shape[0], data_test.shape[1], data_test.shape[2], 1))
# print(f"train data shape: {data_train.shape} \n test data shape: {data_test.shape} ")

train data shape: (233742, 50, 86, 1) 
 test data shape: (41488, 50, 86, 1) 


In [141]:
# take part of data to gather results faster
percentage = 0.10
data_train = data_train[0:int(percentage*data_train.shape[0]),:,:]
data_test = data_test[0:int(percentage*data_test.shape[0]),:,:]
print(f"train data shape: {data_train.shape} \n test data shape: {data_test.shape} ")

train data shape: (23374, 50, 86) 
 test data shape: (4148, 50, 86) 


## Model

In [142]:
input_shape = data_train.shape[1:]
n_features = data_train.shape[2]

In [143]:
input_shape

(50, 86)

In [128]:
# normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=None)
# normalizer.adapt(data_train)

In [146]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=input_shape, return_sequences=True))
#model.add(normalizer)
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(width_window))
model.add(LSTM(64, activation='relu', return_sequences=True))
#model.add(normalizer)
model.add(LSTM(128, activation='relu', return_sequences=True))
#model.add(normalizer)
model.add(TimeDistributed(Dense(n_features)))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.000001)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_71 (LSTM)              (None, 50, 128)           110080    
                                                                 
 lstm_72 (LSTM)              (None, 64)                49408     
                                                                 
 repeat_vector_16 (RepeatVec  (None, 50, 64)           0         
 tor)                                                            
                                                                 
 lstm_73 (LSTM)              (None, 50, 64)            33024     
                                                                 
 lstm_74 (LSTM)              (None, 50, 128)           98816     
                                                                 
 time_distributed_14 (TimeDi  (None, 50, 86)           11094     
 stributed)                                          

In [147]:
tf.debugging.set_log_device_placement(True)

try:
    with tf.device('/GPU'):
        data_train = tf.convert_to_tensor(data_train, dtype=tf.int32)
        model = model
        model.compile(optimizer='adam', loss='mse')
except RuntimeError as e:
    print(e)
history = model.fit(data_train, data_train, epochs=20, batch_size=128,validation_data=(data_test, data_test))

Epoch 1/20
183/183 [==============================] - 41s 214ms/step - loss: 0.3859 - val_loss: 54.7813
Epoch 2/20
183/183 [==============================] - 41s 222ms/step - loss: 0.2656 - val_loss: 1.1793
Epoch 3/20
183/183 [==============================] - 40s 221ms/step - loss: 0.2231 - val_loss: 0.8828
Epoch 4/20
183/183 [==============================] - 41s 222ms/step - loss: 0.2035 - val_loss: 0.8836
Epoch 5/20
183/183 [==============================] - 41s 225ms/step - loss: 0.1896 - val_loss: 0.8735
Epoch 6/20
183/183 [==============================] - 41s 225ms/step - loss: 0.1816 - val_loss: 0.8841
Epoch 7/20
183/183 [==============================] - 41s 226ms/step - loss: 0.1754 - val_loss: 0.8815
Epoch 8/20
183/183 [==============================] - 41s 225ms/step - loss: 0.1703 - val_loss: 0.8750
Epoch 9/20
183/183 [==============================] - 42s 228ms/step - loss: 0.1683 - val_loss: 0.8779
Epoch 10/20
183/183 [==============================] - 44s 238ms/step - 

KeyboardInterrupt: 

In [80]:
y_train = model.predict(data_train)

cost_train = y_train - data_train
print(f"mean: {np.round(tf.math.reduce_mean(cost_train, axis=None, keepdims=False, name=None),3)}")

731/731 [==============================] - 20s 28ms/step


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'mean'

In [95]:
y_test = model.predict(data_test)

cost_test = y_test - data_test
print(f"mean: {np.round(tf.math.reduce_mean(cost_test, axis=None, keepdims=False, name=None),3)}")

130/130 [==============================] - 4s 27ms/step
mean: nan
